In [2]:
import bz2
import numpy as np
import pandas as pd

**First exercise: count the number of lines in Python for each file**

In [4]:
def linesCountBz2(fileName):
    f = bz2.BZ2File(fileName, 'rb')
    nblines = 0
    for line in f:
        nblines = nblines + 1
    print fileName, 'has ', nblines, 'lines'

linesCountBz2('downloads/bookings.csv.bz2')
linesCountBz2('downloads/searches.csv.bz2')


 downloads/bookings.csv.bz2 has  10000011 lines
downloads/searches.csv.bz2 has  20390199 lines


**Second exercise: top 10 arrival airports in the world in 2013 (using the bookings file)**

Arrival airport is the column *arr_port*. It is the IATA code for the airport

To get the total number of passengers for an airport, you can sum the column pax, grouping by arr_port. Note that there is negative pax. That corresponds to cancelations. So to get the total number of passengers that have actually booked, you should sum including the negatives (that will remove the canceled bookings).

Print the top 10 arrival airports in the standard output, including the number of passengers.

Bonus point: Get the name of the city or airport corresponding to that airport (programatically, we suggest to have a look at GeoBases in Github)

Bonus point: Solve this problem using pandas (instead of any other approach) 

In [ ]:
def top10Airports(fileName):
    fields = ['arr_port', 'pax']
    # get a textreader
    tf = pd.read_csv(fileName, chunksize=10000, sep='\s*\^', engine='python', iterator=True, usecols=fields)
    #make a DataFrame
    df = pd.concat(tf, ignore_index=True)
    df.groupby('arr_port').sum()
    df.sort_values('pax',inplace=True, ascending=False)
    print df.head(10)
    
top10Airports('downloads/bookings.csv.bz2')
#chunksize=10000
#tf_chunk = read_csv('downloads/bookings.csv.bz2', chunksize=1000, sep='\s*\^', engine='python', iterator=True, usecols=fields)



**Third exercise: plot the monthly number of searches for flights arriving at Málaga, Madrid or Barcelona**

For the arriving airport, you can use the Destination column in the searches file. Plot a curve for Málaga, another one for Madrid, and another one for Barcelona, in the same figure. Bonus point: Solving this problem using pandas (instead of any other approach)



**Bonus exercise: match searches with bookings**

For every search in the searches file, find out whether the search ended up in a booking or not (using the info in the bookings file). For instance, search and booking origin and destination should match. For the bookings file, origin and destination are the columns dep_port and arr_port, respectively. Generate a CSV file with the search data, and an additional field, containing 1 if the search ended up in a booking, and 0 otherwise. 

**Bonus exercise: write a Web Service**

Wrap the output of the second exercise in a web service that returns the data in JSON format (instead of printing to the standard output). The web service should accept a parameter n>0. For the top 10 airports, n is 10. For the X top airports, n is X.